In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta


In [1]:
# Install necessary libraries
# pip install yfinance yahooquery

from yahooquery import search
import yfinance as yf
import pandas as pd
import datetime
import os

def get_stock_data(company_name):
    # Map company name to ticker symbol
    ticker_symbol = get_ticker_symbol(company_name)
    if not ticker_symbol:
        print(f"Can't get ticker symbol for {company_name}")
        return

    # Fetch data from 2019-01-01 to today
    start_date = '2019-01-01'
    end_date = datetime.datetime.today().strftime('%Y-%m-%d')
    data = yf.download(ticker_symbol, start=start_date, end=end_date)
    
    # Check if data is available
    if data.empty:
        print(f"No data available for {company_name} ({ticker_symbol})")
        return

    # Check if 'Adj Close' column exists; if not, use 'Close' as a fallback
    if 'Adj Close' in data.columns:
        data = data[['Adj Close']]
    elif 'Close' in data.columns:
        data = data[['Close']]
        data.rename(columns={'Close': 'Adj Close'}, inplace=True)
    else:
        print(f"No 'Adj Close' or 'Close' data available for {company_name} ({ticker_symbol})")
        return
    
    # Format date and reset index
    data.reset_index(inplace=True)
    data['Date'] = data['Date'].dt.strftime('%Y-%m-%d')

    # Create folder if it doesn't exist
    folder_path = 'StockPrice'
    os.makedirs(folder_path, exist_ok=True)

    # Save data as CSV with company name in filename
    csv_filename = f"{folder_path}/{company_name}_stock_data.csv"
    data.to_csv(csv_filename, index=False)
    print(f"Data saved to {csv_filename}")

    # Retrieve the most recent adjusted close price if real-time data is unavailable
    ticker = yf.Ticker(ticker_symbol)
    history = ticker.history(period='1d')
    if 'Adj Close' in history.columns:
        current_price = history['Adj Close'].iloc[-1]
    elif 'Close' in history.columns:
        current_price = history['Close'].iloc[-1]
    else:
        current_price = None

    if current_price is not None:
        print(f"{company_name} ({ticker_symbol}) most recent Adj Close: ${current_price}")
    else:
        print(f"No recent 'Adj Close' or 'Close' price available for {company_name} ({ticker_symbol})")

def get_ticker_symbol(company_name):
    # Use yahooquery's search function to get the ticker symbol
    result = search(company_name)
    if 'quotes' in result and len(result['quotes']) > 0:
        # Return the first matching ticker symbol
        ticker_symbol = result['quotes'][0]['symbol']
        return ticker_symbol
    else:
        return None


In [2]:

# Example usage
company_name = 'Singtel'
get_stock_data(company_name)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)